In [1]:
from polls import *
from UKGE import *
import numpy as np

In [2]:
df = pd.read_csv("outputs/resultsclusteredconstituencies.csv")

In [3]:
#Predicting 2019
poll_avg19 = get_weighted_poll_avg(url_19, col_dict=col_dict19)
poll_avg17 = get_weighted_poll_avg(url_17, col_dict=col_dict17)
poll_avg15 = get_weighted_poll_avg(url_15, col_dict=col_dict15)
poll_avg10 = get_weighted_poll_avg(url_10, col_dict=col_dict10)
poll_avg05 = get_weighted_poll_avg(url_05, col_dict=col_dict05)
poll_avg19 = standardise_df(poll_avg19, col_dict19)
poll_avg17 = standardise_df(poll_avg17, col_dict17)
poll_avg15 = standardise_df(poll_avg15, col_dict15)
poll_avg10 = standardise_df(poll_avg10, col_dict10)
poll_avg05 = standardise_df(poll_avg05, col_dict05)



In [4]:

gb19 = get_GB_res(df)
pc_res_19 = convert_to_pc(gb19)
gb17 = get_GB_res(df,yr=2017)
pc_res_17 = convert_to_pc(gb17)
gb15 = get_GB_res(df,yr=2015)
pc_res_15 = convert_to_pc(gb15)
gb10 = get_GB_res(df,yr=2010)
pc_res_10 = convert_to_pc(gb10)
gb05 = get_GB_res(df,yr=2005)
pc_res_05 = convert_to_pc(gb05)

In [5]:
poll_avgs = [poll_avg05,poll_avg10,poll_avg15,poll_avg17,poll_avg19]
pc_results = [pc_res_05,pc_res_10,pc_res_15,pc_res_17,pc_res_19]

In [6]:
bigerrs = []
for p,r in zip(poll_avgs,pc_results):
    conerr = float(p["con"] - r["con"])
    laberr = float(p["lab"] - r["lab"])

    bigerrs.append(conerr)
    bigerrs.append(laberr)






/tmp/ipykernel_12353/1519193469.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  conerr = float(p["con"] - r["con"])
/tmp/ipykernel_12353/1519193469.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  laberr = float(p["lab"] - r["lab"])


In [7]:
big_std = np.std(bigerrs)

In [8]:
pc_res_05

,con,lab,lib,ref,nat,oth
0,0.332274,0.36199,0.226456,0.0,0.022213,0.057067


In [9]:
pd.DataFrame(poll_avg05).transpose().columns

Index(['Con', 'Lab', 'Lib Dem', 'Others', 'con', 'lab', 'lib', 'oth'], dtype='object')

In [10]:
serrs = []
for p,r in zip(poll_avgs,pc_results):
    liberr = float(p["lib"] - r["lib"])
    serrs.append(liberr)
    if "ref" in pd.DataFrame(p).transpose().columns:
        referr = float(p["ref"] - r["ref"])
        serrs.append(referr)
        
    if "nat" in pd.DataFrame(p).transpose().columns:
        naterr = float(p["nat"] - r["nat"])
        serrs.append(naterr)




/tmp/ipykernel_12353/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_12353/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_12353/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_12353/2919726663.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  referr = float(p["ref"] - r["ref"])
/tmp/ipykernel_12353/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a

In [11]:
small_std = np.std(serrs)

In [12]:
constituencies = list(df["New constituency name"])

In [13]:
next_poll_avg = get_weighted_poll_avg(next_url, col_dict=next_col_dict)
nat_polls = standardise_df(next_poll_avg, next_col_dict)
chg_df = nat_polls - pc_res_19
chg_df = chg_df.dropna(axis=1).copy()

In [14]:
chg_df

,con,lab,lib,nat,oth,ref
0,-0.225489,0.109106,-0.03195,-0.015484,0.016707,0.086444


In [15]:
natchgs = []
sims_num = 100
future_uncertainty = 3
for p in std_parties:
    if nat_polls[p] > 0.2:
        natchgs.append(np.random.normal(chg_df[p],big_std*future_uncertainty,sims_num))
    else:
        natchgs.append(np.random.normal(chg_df[p],small_std*future_uncertainty,sims_num))
clusters = list(set(list(df["Cluster"])))
for p in std_parties:
    for c in clusters:
        if nat_polls[p] > 0.2:
            natchgs.append(np.random.normal(0,big_std,sims_num))
        else:
            natchgs.append(np.random.normal(0,small_std,sims_num))

In [16]:
chg_dict = {}


In [17]:
i=0
for p in std_parties:
    chg_dict[p+"n"] = natchgs[i]
    i+=1
for c in clusters:
    for p in std_parties:
        chg_dict[p+str(c)] = natchgs[i]
        i+=1

In [18]:
rand_df = pd.DataFrame(chg_dict)

In [19]:
# Create a dictionary to store the results for each constituency and party
results = {}

# Iterate over constituencies
for con in constituencies:
    # Filter the dataframe for the current constituency
    con_df = df[df["New constituency name"] == con].copy()
    
    # Get the cluster, imputation uncertainty, and future uncertainty for the current constituency
    cluster = con_df["Cluster"].iloc[0]
    imputation_uncertainty = con_df["Low_Confidence_Imputation"].iloc[0]
    
    # Iterate over standard parties
    for p in std_parties:
        # Get the previous results for the current party and constituency
        prev_res = con_df["2019_Results"+p+"_pc"]
        
        # Get the necessary data from the random dataframe
        nat_chg = rand_df[p+"n"]
        clr_chg = rand_df[p+str(cluster)]
        
        # Calculate the noise for the current constituency and party
        con_noise = [np.random.normal(0, (1+imputation_uncertainty)*small_std) for n in range(sims_num)]
        
        # Calculate the rough prediction for the current constituency and party
        prev_res_array = [float(prev_res.iloc[0]) for n in range(sims_num)]
        rough_pred = prev_res_array + nat_chg + clr_chg + con_noise
        rough_pred[rough_pred < 0] = 0
        
        # Store the results in the dictionary
        results[con+p+"Noise"] = con_noise
        results[con+p+"Rough"] = rough_pred
    
# Update the random dataframe with the results
rand_df = pd.concat([rand_df,pd.DataFrame(results)],axis=1)

In [20]:
def get_winner(party_values, winner_val,parties = std_parties):
    for p,v in zip(parties,list(party_values)[0]):
        if v == winner_val:
            return p
def get_margin(party_values):
    sorted_vals = sorted(list(party_values)[0])
    margin = sorted_vals[-1] - sorted_vals[-2]
    return margin

final_dfs = [rand_df]
for con in constituencies:
    con_cols = []
    for p in std_parties:
        con_cols.append(con+p+"Rough")
    final_df = rand_df[con_cols].copy()
    final_pred_cols = []
    for p in std_parties:
        final_df[con+p+"Final"] = final_df[con+p+"Rough"] / final_df[con_cols].sum(axis=1)
        final_pred_cols.append(con+p+"Final")
    final_df[con +"past_post_val"] = final_df[final_pred_cols].max(axis=1)
    final_df[con+"Margin"] = final_df[final_pred_cols].apply(lambda x:get_margin([x[final_pred_cols]]),axis=1)
    final_pred_cols.append(con +"past_post_val")
    final_df[con+"Winner"] = final_df[final_pred_cols].apply(lambda x:get_winner([x[final_pred_cols]],x[con+"past_post_val"]),axis=1)

                
    # final_df[con+"Winner"] = winners
    final_dfs.append(final_df)
rand_df = pd.concat(final_dfs, axis=1)
    

In [21]:
seats_cols = []
for p in std_parties:
    rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
    seats_cols.append(p+"Seats")

/tmp/ipykernel_12353/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_12353/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_12353/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

In [22]:
rand_df[seats_cols]

,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats
0,221,318,36,1,55,1
1,245,326,18,1,41,1
2,82,423,57,7,58,5
3,168,400,19,0,44,1
4,281,290,7,0,53,1
...,...,...,...,...,...,...
95,212,315,37,5,58,5
96,223,358,16,0,34,1
97,211,339,31,0,50,1
98,74,493,34,0,29,2


In [23]:
max_seats = rand_df[seats_cols].max(1)
results = []
for n in range(sims_num):
    for col in seats_cols:
        if max_seats[n] ==rand_df[col].loc[n]:
            for p in std_parties:
                if p + "Seats" == col:
                    winning_party = p
    
    if max_seats[n] <326:
        result = "Hung Parliament: Largest Party " + winning_party
    else:
        result = winning_party
    results.append(result)
rand_df["Result"] = results


/tmp/ipykernel_12353/3465883602.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df["Result"] = results


In [24]:
rand_df["Result"].value_counts()

Result
lab                                   84
Hung Parliament: Largest Party lab    14
Hung Parliament: Largest Party con     2
Name: count, dtype: int64

In [25]:
med_df = rand_df[(rand_df["conSeats"]>rand_df["conSeats"].quantile(0.49))&(rand_df["conSeats"]>rand_df["conSeats"].quantile(0.49))]

In [26]:
typical_run = med_df[(med_df["labSeats"]>med_df["labSeats"].quantile(0.4))&(med_df["labSeats"]<med_df["labSeats"].quantile(0.6))].copy()

In [27]:
typical_run

,conn,labn,libn,refn,natn,othn,con0,lab0,lib0,ref0,...,West Dunbartonshirepast_post_val,West DunbartonshireMargin,West DunbartonshireWinner,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats,Result
7,-0.162031,0.124034,-0.074584,0.065533,-0.074650,0.053223,0.004576,0.024517,0.008972,-0.004813,...,0.460022,0.099665,nat,238,337,14,0,42,1,lab
21,-0.269289,0.040100,-0.033576,0.086289,-0.039218,0.075319,0.011774,0.008085,-0.015557,-0.013913,...,0.510754,0.203320,nat,204,337,32,1,54,4,lab
54,-0.172494,0.081828,-0.025751,0.145682,-0.059336,0.041493,0.022984,-0.028706,-0.002512,0.014792,...,0.432978,0.096858,nat,224,337,24,1,44,2,lab
58,-0.196443,0.076354,0.000917,0.128364,-0.016762,0.127352,-0.004632,-0.035638,-0.018209,0.000647,...,0.391530,0.056550,nat,218,333,28,0,49,4,lab
59,-0.210233,0.066852,-0.037411,0.059333,0.020166,0.041282,-0.016131,-0.057163,0.012109,-0.016227,...,0.537935,0.162380,nat,217,339,18,0,56,2,lab
63,-0.160482,0.115677,-0.034082,0.072200,0.073867,0.047921,-0.003118,-0.003554,0.009268,0.014958,...,0.560104,0.214285,nat,226,330,19,0,56,1,lab
71,-0.223626,0.070814,-0.079875,0.031027,-0.006858,0.003735,-0.005831,-0.011944,0.002041,-0.013244,...,0.561276,0.210183,nat,224,332,20,0,55,1,lab
88,-0.164395,0.123836,-0.028401,0.073477,0.049837,0.016299,0.013499,0.005452,-0.011688,-0.021764,...,0.504479,0.122290,nat,221,332,19,0,59,1,lab
97,-0.189486,0.102108,0.008599,0.148029,-0.014424,0.064671,0.040202,-0.034176,-0.029534,-0.005792,...,0.410005,0.088187,nat,211,339,31,0,50,1,lab


In [28]:
print(nat_polls["lab"] - nat_polls["con"])

0.21383333333333324


In [29]:
nat_polls

Con         0.234500
Lab         0.448333
Lib Dems    0.089833
SNP         0.030500
Green       0.061167
Reform      0.107667
Others      0.028500
con         0.234500
lab         0.448333
lib         0.089833
ref         0.107667
nat         0.030500
oth         0.028500
dtype: float64

In [30]:
winner_cols = [con+"Winner" for con in constituencies]

In [31]:
winner_wins = []
for col in winner_cols:
    winner_wins.append(max(list(rand_df[col].value_counts())))

In [32]:
close_cols = []
for col,wins in zip(winner_cols,winner_wins):
    if wins < 525:
        close_cols.append(col)

In [33]:
rand_df[close_cols]

,Aberafan MaestegWinner,AldershotWinner,Aldridge-BrownhillsWinner,Altrincham and Sale WestWinner,Alyn and DeesideWinner,Amber ValleyWinner,Arundel and South DownsWinner,AshfieldWinner,AshfordWinner,Ashton-under-LyneWinner,...,North Ayrshire and ArranWinner,North East FifeWinner,Orkney and ShetlandWinner,Paisley and Renfrewshire NorthWinner,Paisley and Renfrewshire SouthWinner,Perth and Kinross-shireWinner,RutherglenWinner,Stirling and StrathallanWinner,West Aberdeenshire and KincardineWinner,West DunbartonshireWinner
0,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,lib,lib,nat,nat,nat,nat,nat,nat,nat
1,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,lib,lib,nat,nat,nat,lab,nat,nat,nat
2,lab,lab,con,lab,lab,lab,con,oth,lab,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
3,lab,lab,con,lab,lab,lab,con,lab,lab,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
4,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,nat,nat,nat,nat,lab,nat,nat,nat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,lab,con,con,lab,lab,con,con,oth,con,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
96,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,lib,nat,nat,nat,lab,nat,nat,nat
97,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
98,lab,lab,lab,lab,lab,lab,con,lab,lab,lab,...,nat,lib,lib,lab,lab,nat,lab,nat,nat,lab


In [39]:
all_final_cols = [con+p+"Final"for p in std_parties for con in constituencies]
all_margin_cols = [con+"Margin"for p in std_parties for con in constituencies]
winner_cols
results_cols = ["Result"]

In [40]:
all_result_cols = []
for l in [all_final_cols,all_margin_cols,winner_cols,results_cols]:
    all_result_cols.extend(l)

In [41]:
rand_df[all_result_cols]

,Aberafan MaestegconFinal,AldershotconFinal,Aldridge-BrownhillsconFinal,Altrincham and Sale WestconFinal,Alyn and DeesideconFinal,Amber ValleyconFinal,Arundel and South DownsconFinal,AshfieldconFinal,AshfordconFinal,Ashton-under-LyneconFinal,...,North East FifeWinner,Orkney and ShetlandWinner,Paisley and Renfrewshire NorthWinner,Paisley and Renfrewshire SouthWinner,Perth and Kinross-shireWinner,RutherglenWinner,Stirling and StrathallanWinner,West Aberdeenshire and KincardineWinner,West DunbartonshireWinner,Result
0,0.000000,0.383353,0.462547,0.283437,0.171815,0.469118,0.394300,0.153297,0.497707,0.017772,...,lib,lib,nat,nat,nat,nat,nat,nat,nat,Hung Parliament: Largest Party lab
1,0.090316,0.412676,0.494589,0.306074,0.219357,0.479893,0.431055,0.203944,0.439385,0.148797,...,lib,lib,nat,nat,nat,lab,nat,nat,nat,lab
2,0.000000,0.241112,0.351441,0.100725,0.025100,0.308769,0.262771,0.007598,0.332452,0.000000,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,lab
3,0.092893,0.453809,0.547022,0.326600,0.232297,0.489524,0.412249,0.229440,0.465801,0.202091,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,lab
4,0.149962,0.543495,0.599596,0.395366,0.323979,0.546790,0.543620,0.277305,0.548128,0.235778,...,nat,nat,nat,nat,nat,lab,nat,nat,nat,Hung Parliament: Largest Party lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.002467,0.358040,0.438399,0.203730,0.139364,0.399768,0.358951,0.136904,0.344200,0.080622,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,Hung Parliament: Largest Party lab
96,0.112881,0.415901,0.500298,0.328872,0.236754,0.457573,0.432624,0.217336,0.403537,0.226966,...,nat,lib,nat,nat,nat,lab,nat,nat,nat,lab
97,0.082441,0.378103,0.451633,0.281587,0.205562,0.401654,0.369068,0.178204,0.482713,0.176426,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,lab
98,0.011324,0.320019,0.371186,0.230808,0.124263,0.368519,0.347361,0.118598,0.348741,0.084943,...,lib,lib,lab,lab,nat,lab,nat,nat,lab,lab


In [38]:
all_margin_cols

['Aberafan MaestegconMargin',
 'AldershotconMargin',
 'Aldridge-BrownhillsconMargin',
 'Altrincham and Sale WestconMargin',
 'Alyn and DeesideconMargin',
 'Amber ValleyconMargin',
 'Arundel and South DownsconMargin',
 'AshfieldconMargin',
 'AshfordconMargin',
 'Ashton-under-LyneconMargin',
 'AylesburyconMargin',
 'BanburyconMargin',
 'Bangor AberconwyconMargin',
 'BarkingconMargin',
 'Barnsley NorthconMargin',
 'Barnsley SouthconMargin',
 'Barrow and FurnessconMargin',
 'Basildon and BillericayconMargin',
 'BasingstokeconMargin',
 'BassetlawconMargin',
 'BathconMargin',
 'BatterseaconMargin',
 'BeaconsfieldconMargin',
 'Beckenham and PengeconMargin',
 'BedfordconMargin',
 'Bermondsey and Old SouthwarkconMargin',
 'Bethnal Green and StepneyconMargin',
 'Beverley and HoldernessconMargin',
 'Bexhill and BattleconMargin',
 'Bexleyheath and CrayfordconMargin',
 'Bicester and WoodstockconMargin',
 'BirkenheadconMargin',
 'Birmingham EdgbastonconMargin',
 'Birmingham ErdingtonconMargin',
 'Bi